### Import Libraries

In [41]:
import pandas as pd
a=pd.read_json('data_twitter/wholedata/train.json',lines=True)
b=pd.read_json('data_twitter/wholedata/dev.json',lines=True)
c=pd.read_json('data_twitter/wholedata/eval.json',lines=True)
train_l=pd.DataFrame(a.sentiment/4).astype(int)
dev_l=pd.DataFrame(b.sentiment/4).astype(int)
eval_l=pd.DataFrame(c.sentiment/4).astype(int)

train_l.to_csv('data_twitter/wholedata/train.labels',header=None,index=False)
a[['tweet']].to_csv('data_twitter/wholedata/train.data',header=None,index=False)
dev_l.to_csv('data_twitter/wholedata/dev.labels',header=None,index=False)
b[['tweet']].to_csv('data_twitter/wholedata/dev.data',header=None,index=False)
eval_l.to_csv('data_twitter/wholedata/eval.labels',header=None,index=False)
c[['tweet']].to_csv('data_twitter/wholedata/eval.data',header=None,index=False)

In [22]:
train_l

,sentiment
0,0
1,1
2,0
3,0
4,0
...,...
9995,0
9996,1
9997,1
9998,1


In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [1]:
from collections import OrderedDict
import torch

In [2]:
import sys
sys.path.append("sopa_master/")

In [3]:
import tensorboardX
from data import read_embeddings, read_docs, read_labels

from soft_patterns import ProbSemiring, MaxPlusSemiring, LogSpaceMaxTimesSemiring, SoftPatternClassifier, train, Batch, evaluate_accuracy
from util import to_cuda
from interpret_classification_results import interpret_documents
from visualize import visualize_patterns
import pickle

### Files

In [4]:
logfile="sopa_master/"
embedding_file =logfile+ "glove.twitter.27B.200d.txt"
train_data_file = "data_twitter/wholedata/train.data"
train_label_file ="data_twitter/wholedata/train.labels"
dev_data_file = "data_twitter/wholedata/dev.data"
dev_label_file = "data_twitter/wholedata/dev.labels"
test_data_file = "data_twitter/wholedata/eval.data"
test_label_file="data_twitter/wholedata/eval.labels"

### Model's parameters

In [5]:
patterns = "5-5_4-5_3-5_2-5"
pattern_specs = OrderedDict(sorted(([int(y) for y in x.split("-")] for x in patterns.split("_")),
                                key=lambda t: t[0]))

In [6]:
pattern_specs

OrderedDict([(2, 5), (3, 5), (4, 5), (5, 5)])

In [7]:
num_padding_tokens=max(list(pattern_specs.keys())) - 1
num_padding_tokens

4

vocab, embeddings, word_dim = read_embeddings(embedding_file)
pickle.dump(vocab, open("twitter_vocab_200.p","wb"))
pickle.dump(embeddings, open("twitter_embeddings_200.p","wb"))
pickle.dump(word_dim, open("twitter_word_dim_200.p","wb"))


In [8]:

vocab = pickle.load(open("twitter_vocab_200.p","rb"))
embeddings = pickle.load(open("twitter_embeddings_200.p","rb"))
word_dim = pickle.load(open("twitter_word_dim_200.p","rb"))

In [15]:
model = SoftPatternClassifier(
    pattern_specs=pattern_specs,
    mlp_hidden_dim=10,
    num_mlp_layers=2,
    num_classes=2,
    embeddings=embeddings,
    vocab=vocab,
    semiring=LogSpaceMaxTimesSemiring,
    bias_scale_param=1,
    shared_sl=False,
    no_sl=False
)

20 OrderedDict([(2, 5), (3, 5), (4, 5), (5, 5)])
# params: 40512


### Training

In [9]:
train_input, train_text = read_docs(train_data_file, vocab, num_padding_tokens=num_padding_tokens)
train_labels = read_labels(train_label_file)
dev_input, dev_text = read_docs(dev_data_file, vocab, num_padding_tokens=num_padding_tokens)
dev_labels = read_labels(dev_label_file)

In [10]:
test_input, test_text = read_docs(test_data_file, vocab, num_padding_tokens=num_padding_tokens)

In [11]:
test_label = read_labels(test_label_file)
test_data = list(zip(test_input, test_label))

In [12]:
train_data = list(zip(train_input, train_labels))
dev_data = list(zip(dev_input, dev_labels))

In [13]:
len(train_data)

1274400

In [16]:
train(
    train_data=train_data,
    dev_data=dev_data,
    model=model,
    model_save_dir="model_twitter/",
    num_iterations=30,
    model_file_prefix="experiment200_1",
    learning_rate=0.005,
    batch_size=150,
    num_classes=2,
    patience=5
)

..

sopa_master/soft_patterns.py:505: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_softmax(output).view(batch.size(), num_classes),


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

...............................................................................................................................................................................................................................................................................................................................................

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



........................

num predicted 1s: 517
num gold 1s:      502
precision 0.8181818181818182
recall 0.8426294820717132
f1 score 0.830225711481845
num predicted 1s: 83889
num gold 1s:      82947
precision 0.8086638295843317
recall 0.8178475412010079
f1 score 0.8132297585652977
iteration:      29 train time:   459.979m, eval time:     0.757m train loss:        0.390 train_acc:   82.700% dev loss:        0.410 dev_acc:   81.184%


SoftPatternClassifier(
  (mlp): MLP(
    (layers): ModuleList(
      (0): Linear(in_features=20, out_features=10, bias=True)
      (1): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

In [17]:
torch.save(model.state_dict(), "model_twitter/experiment200_1.pth")

In [19]:
torch.save(model, "model_twitter/experiment2002.pth")

PicklingError: Can't pickle <function <lambda> at 0x7f6daf329e18>: attribute lookup <lambda> on soft_patterns failed

### Forward

In [18]:
evaluate_accuracy(model, test_data, batch_size=150, gpu=False)

num predicted 1s: 81022
num gold 1s:      79965
precision 0.8082125842363802
recall 0.8188957668980179
f1 score 0.8135191040270331


0.81236875

In [15]:
model_dict = torch.load("model_twitter/experiment_6_49.pth")

In [16]:
model.load_state_dict(model_dict)

In [ ]:
interpret_documents(model, 1, test_data, test_text,"model_twitter/experiment200.txt", -1)

In [ ]:
visualize_patterns(model,dev_set=test_data,dev_text=test_text,k_best=5,max_doc_len=-1,num_padding_tokens=0)

In [ ]:
model.get_transition_matrices(batch_obj)[0][0, :, :, :].data[:1,1,:-1]

In [ ]:
batch = Batch([doc[0]], model.embeddings, model.to_cuda, 0, max_doc_len) 

In [30]:
visualize_patterns(model,dev_set=test_data,dev_text=test_text,k_best=5,max_doc_len=-1,num_padding_tokens=0)

.Pattern: 0 of length 2
Highest scoring spans:
0 -0.064  b'HP uncompromising #label=1'
1 -0.070  b'HP spirited       #label=1'
2 -0.090  b'HP thoughtful     #label=1'
3 -0.093  b'HP endearing      #label=1'
4 -0.093  b'HP endearing      #label=1'
self-loops:   1.00 * staters         +  1.32,  1.00 * May             + -2.47
fwd 1s:       6.40 * uncompromising  +  0.31
epsilons:                              -0.87

Pattern: 1 of length 2
Highest scoring spans:
0 -0.124  b'HP insipid        #label=0'
1 -0.155  b'HP flailing       #label=0'
2 -0.156  b'HP bland          #label=0'
3 -0.156  b'HP bland          #label=0'
4 -0.156  b'HP dull           #label=0'
self-loops:   1.00 * sux             + -2.29,  1.00 * 144,00          + -2.21
fwd 1s:       7.23 * uninspiring     + -0.50
epsilons:                              -0.73

Pattern: 2 of length 2
Highest scoring spans:
0 -0.081  b'HP feel-bad       #label=0'
1 -0.085  b'HP movie-esque    #label=0'
2 -0.089  b'HP repetitively   #label=0'
3 -